<a href="https://colab.research.google.com/github/ayoni02/twitter-x-sentiment/blob/main/Sentiment%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

# Reading the CSV file
To download it, go to https://www.kaggle.com/datasets/kazanova/sentiment140

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ALx/training.csv', encoding='latin-1', header=None, names=['target', 'id', 'date', 'flag', 'user', 'text'])

In [ ]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df = df.drop(['id', 'date', 'flag', 'user'], axis=1)

In [ ]:
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
df= df.sample(frac= 0.03, random_state= 42)

In [ ]:
df['target'].value_counts()

4    24007
0    23993
Name: target, dtype: int64

# Building Model

I used a pipeling so I can train it all in one go

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = stopwords.words('english')


This is meant to clean up each tweet

In [ ]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()

    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

    return [word for word in nopunc.split() if word.lower() not in stop_words]

In [ ]:
"""
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
#df['text'] = df['text'].apply(lambda x: preprocess(x))
"""

'\n    for token in text.split():\n        if token not in stop_words:\n            if stem:\n                tokens.append(stemmer.stem(token))\n            else:\n                tokens.append(token)\n    return " ".join(tokens)\n#df[\'text\'] = df[\'text\'].apply(lambda x: preprocess(x))\n'

In [ ]:
df.head()

,target,text
541200,0,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,@TiannaChaos i know just family drama. its la...
285055,0,School email won't open and I have geography ...
705995,0,upper airways problem


In [ ]:
X = df['text']
y = df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Training the pipeline model

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=preprocess)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function preprocess at 0x7bc049f8eb00>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [ ]:
predict = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predict,y_test))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74      8564
           4       0.69      0.75      0.72      7276

    accuracy                           0.73     15840
   macro avg       0.73      0.73      0.73     15840
weighted avg       0.74      0.73      0.73     15840



# Saving the model
This is so i can use it in my script directly

In [ ]:
import joblib

In [ ]:
joblib.dump(pipeline, '/content/drive/MyDrive/ALx/pipeline.pkl')

['/content/drive/MyDrive/ALx/pipeline.pkl']